# Assignment 1

### Joey Schnecker PID: A12028163
---

In [91]:
import time

#### Import and Load the Base Overlay to the PL

In [92]:
from pynq.overlays.base import BaseOverlay
base = BaseOverlay("base.bit")

#### PMOD Pinout
![PMOD Pinout](images/pmod_pinout.png)

#### Modified Lab 1 Code

`clear_gpio()` function was added

In [93]:
%%microblaze base.PMODB

#include "gpio.h"
#include "pyprintf.h"

//Function to turn on/off a selected pin of PMODB
void write_gpio(unsigned int pin, unsigned int val){
    if (val > 1){
        pyprintf("pin value must be 0 or 1");
    }
    gpio pin_out = gpio_open(pin);
    gpio_set_direction(pin_out, GPIO_OUT);
    gpio_write(pin_out, val);
    //gpio_close(pin_out);
}

//Function to read the value of a selected pin of PMODB
unsigned int read_gpio(unsigned int pin){
    gpio pin_in = gpio_open(pin);
    gpio_set_direction(pin_in, GPIO_IN);
    return gpio_read(pin_in);
}

//Function that resets all PMODB Pins
void clear_gpio(){
    for(int i=0; i<=7; i++){
        gpio pin = gpio_open(i);
        gpio_close(pin);
        write_gpio(i, 0);
        //gpio pin = gpio_open(i);
        //gpio_write(pin, 0);
        //gpio_close(pin);
    }
}

#### Python Code to Emulate PWM on GPIO Pin

In [94]:
#Test cell to turn on LED
write_gpio(1,0)
write_gpio(2,1)
write_gpio(3,0)

In [95]:
clear_gpio()

In [96]:
# def turn_off():
#     write_gpio(1,0)
#     write_gpio(2,0)
#     write_gpio(3,0)

#this function breaks if left running for too long at higher frequencies
def pwm_gpio(pin,freq,dc,count=-1):
    if pin < 0 or pin > 7:
        print("Invalid Pin")
        return
    if count == 0 or count < -1:
        print("Invalid count")
        return #do nothing
    if dc == 0:
        write_gpio(pin,0)
        print("0% Duty Cycle")
        return 
    if dc == 1:
        write_gpio(pin,1)
        print("100% Duty Cycle")
        return  
    per = round(1.0/freq,5)
    t_on = round(dc * per,5)
    t_off = round(per - t_on,5)
    print("Period:   {}".format(per))
    print("Time On:  {}".format(t_on))
    print("Time Off: {}".format(t_off))
    i = 0
    while i < abs(count):
        write_gpio(pin,1)
        time.sleep(t_on)
        write_gpio(pin,0)
        time.sleep(t_off)
        if count != -1: 
            i += 1
    print("Blinked {} times".format(i))
            

In [102]:
pin = 1
freq = 45 #Hz
dc = 0.25
#duty cycle 
cycles = 200
pwm_gpio(pin,freq,dc,cycles)

Period:   0.02222
Time On:  0.00556
Time Off: 0.01666
Blinked 200 times


In [51]:
clear_gpio()

In [69]:
leds = {
    'red':3,
    'green':2,
    'blue':1
}

## Comparison of Pmod_PWM vs Pmod_IO python classes

After experimenting using the Pmod_PWM you can set a PMOD pin to be used as a pwm generator, but you give up control as gpio. Once a Pmod_IO or Pmod_PWM instance is assigned to PMODB you cannot assign any other function to it. So for example you cannot use Pmod_PWM to make a pwm generator on pin1 and then also use Pmod_IO to read and write to pin1 as gpio (unless you reaload the base overlay). 

Usind the Pmod_PWM class you get get us level control over the period of the PWM generator. Since the PWM is generated on the microblaze and is just being called by the pyton wrapper, this pwm generator can be very precise (us) timing wise. 

Comparatively, using the Pmod_IO class lets you configure PMOD B as general gpio, and then you can create a custom pwm generation fucntion in python. This lets you generate a pwm and also read/write the gpio pins whenever you want. The downside is that the PWM waveform has less percise timing than using the Pmod_PWM class(eg maybe a 51% dc instead of 50% or 10.2Hz instead of 10Hz). 

For this application the Pmod_IO approach is preffered becasue it is more flexable and we do not need the accuarcy that the Pmod_PWM might provide. 

### Testing the pynq.lib pwm module
Pmod_PWM Documentation: https://pynq.readthedocs.io/en/v2.1/pynq_package/pynq.lib/pynq.lib.pmod.html#module-pynq.lib.pmod.pmod_pwm

In [267]:
import time
from pynq.lib import Pmod_PWM
from pynq.overlays.base import BaseOverlay
base = BaseOverlay("base.bit")

In [143]:
freq = 45
period = round(1.0/freq,6)#round to us 
period_us = int(period * 1e6)
if period_us > 65536: #note this is about 16hz 
    print("Warning pwm.generate() wont work!")
pin = leds['blue']
dc = 90
print("Period (s)  = {}".format(period))
print("Period (us) = {}".format(period_us))

Period (s)  = 0.022222
Period (us) = 22222


In [144]:
# Select PMOD (PMODA or PMODB) and output pin (0 -7)
pwm = Pmod_PWM(base.PMODB,pin)
# Set period in us and duty cycle
pwm.generate(period_us,dc)
# Stop PWM
# pwm.stop()

In [145]:
pwm.stop()

### Testing Pmod_IO python class

Pmod_IO Documentation:
https://pynq.readthedocs.io/en/v2.1/pynq_package/pynq.lib/pynq.lib.pmod.html#module-pynq.lib.pmod.pmod_io

In [80]:
import time
from pynq.lib import Pmod_IO
from pynq.overlays.base import BaseOverlay
base = BaseOverlay("base.bit")

In [81]:
#creat Pmos_IO objects for all used pins
gpio_pin_1 = Pmod_IO(base.PMODB,1,'out')
gpio_pin_2 = Pmod_IO(base.PMODB,2,'out')
gpio_pin_3 = Pmod_IO(base.PMODB,3,'out')

#store thses pins in a dictionary to easily refrence based on color
pins = {
    'red'   : gpio_pin_3, 
    'green' : gpio_pin_2,
    'blue'  : gpio_pin_1
}   

In [82]:
pins['blue'].write(1) #turns on blue led

In [85]:
def clear_gpio():
    for color in pins:
        pins[color].write(0)

In [86]:
clear_gpio() #turns off all leds

In [8]:
def generate_pwm(color,freq,dc,cycles,supress_output=True):
    if not((color == 'red') or (color == "blue") or (color == 'green')):
        print("Invalid input. Enter 'red', 'green or 'blue' as first arg.")
        return 
    if dc < 1:
        pins[color].write(0)
        print('dc < 1, write 0 to gpio pin.')
        return 
    if dc > 99:
        pins[color].write(1)
        print('dc > 99, write 1 to gpio pin.')
        return 
    if cycles < 1:
        print("Cycles must be >= 1.")
        return 
    period = round(1.0/freq,6) #round period to nearest us
    ton = round(0.01*dc*period,6)
    toff = round(period-ton,6)
    for i in range(0,cycles):
        pins[color].write(1)
        time.sleep(ton)
        pins[color].write(0)
        time.sleep(toff)
    if not(supress_output):
        print("Period:   {}".format(period))
        print("Time On:  {}".format(ton))
        print("Time Off: {}".format(toff))
        print("done!")

In [87]:
def calc_cycles(freq,desired_time):
    period = round(1.0/freq,6)
    cycles = int(desired_time/period)
    return cycles

In [28]:
freq = 60
dc = 50
color = "blue"
run_time = 1
cycles = calc_cycles(freq,run_time)
print(cycles)
start_time = time.time()
generate_pwm(color,freq,dc,cycles,False)
print("Run Time {} seconds".format(round(time.time()-start_time,3)))

59
Period:   0.016667
Time On:  0.008334
Time Off: 0.008333
done!
Run Time 1.033 seconds


#### Test to see Dimming

In [27]:
for dc in range(0,100):
   generate_pwm(color,freq,dc,cycles) 

dc < 1, write 0 to gpio pin.


### Async PWM Generator
let the program do other stuff while the pwm is being gnerated

In [281]:
async def async_generate_pwm(supress_output=True):
    global freq,dc,color,gen_pwm
    if not((color == 'red') or (color == "blue") or (color == 'green')):
        print("Invalid input. Enter 'red', 'green or 'blue' as first arg.")
        return 
    if dc < 1:
        pins[color].write(0)
        print('dc < 1, write 0 to gpio pin.')
        return 
    if dc > 99:
        pins[color].write(1)
        print('dc > 99, write 1 to gpio pin.')
        return 
    if cycles < 1:
        print("Cycles must be >= 1.")
        return 
    while gen_pwm:
        period = round(1.0/freq,6) #round period to nearest us
        ton = round(0.01*dc*period,6)
        toff = round(period-ton,6)
        pins[color].write(1)
        await asyncio.sleep(ton)
        pins[color].write(0)
        await asyncio.sleep(toff)
    if not(supress_output):
        print("Period:   {}".format(period))
        print("Time On:  {}".format(ton))
        print("Time Off: {}".format(toff))
        print("done!")

### Part 6

In [88]:
import asyncio

In [89]:
freq = 60
dc = 25
color = 'red'
gen_pwm = True
iterations = 59
pause_time = 1
btns = base.btns_gpio
run = True

In [90]:
def calc_cycles(freq,desired_time):
    period = round(1.0/freq,6)
    cycles = int(desired_time/period)
    return cycles

In [54]:
# async def flash_leds():
#     global color, freq, dc, iterations
#     while True:
#         print("Flashing LED")
#         generate_pwm(color,freq,dc,iterations) 
#         await asyncio.sleep(1.0)

In [73]:
async def flash_leds(_loop):
    global color, freq, dc, iterations, pause_time, run
    while True:
        print("Flashing LED")
#         if not((color == 'red') or (color == "blue") or (color == 'green')):
#             print("Invalid input. Enter 'red', 'green or 'blue' as first arg.")
#             _loop.stop()
#             return 
#         if dc < 1:
#             pins[color].write(0)
#             print('dc < 1, write 0 to gpio pin.')
#             return 
#         if dc > 99:
#             pins[color].write(1)
#             print('dc > 99, write 1 to gpio pin.')
#             return 
#         if cycles < 1:
#             print("Cycles must be >= 1.")
#             return 
        period = round(1.0/freq,6) #round period to nearest us
        ton = round(0.01*dc*period,6)
        toff = round(period-ton,6)
        clear_gpio()
        for i in range(0,iterations):
            pins[color].write(1)
            await asyncio.sleep(ton)
            pins[color].write(0)
            await asyncio.sleep(toff)
        await asyncio.sleep(pause_time)
        print("Pausing")
#     if not(supress_output):
#         print("Period:   {}".format(period))
#         print("Time On:  {}".format(ton))
#         print("Time Off: {}".format(toff))
#         print("done!")

In [74]:
async def get_btns(_loop):
    global color, btns
    while True:
        await asyncio.sleep(0.01)
        btn_press = btns.read()
        if btn_press == 1:
            color = 'red'
            print("Button 0 Pressed")
        elif btn_press == 2:
            color = 'blue'
            print("Button 1 Pressed")
        elif btn_press == 4:
            color = 'green'
            print("Button 2 Pressed")
        elif btn_press == 8:
            _loop.stop()
            print("Button 3 Pressed")
            cond = False
        elif btn_press != 0:
            print(btn_press)

In [77]:
loop = asyncio.new_event_loop()
loop.create_task(flash_leds(loop))
loop.create_task(get_btns(loop))
loop.run_forever()
loop.close()        
print("Done.")

KeyboardInterrupt: 